In [50]:
%run 'Setup.py'

8 different classes: Electronic, Experimental, Folk, Hip-Hop, Instrumental, International, Pop or Rock.
objective 1: construct a classifier which, based on the features of a song, predicts its genre
objective 2: estimate its generalisation error under the 0–1 loss.
Features are real-valued, correspond to summary statistics (mean, sd, skewness, kurtosis, median, min, max) of 
time series of various music features, such as the chromagram or the Mel-frequency cepstrum.
Feature description: 

Feature description: 
chroma_cens: Chroma Energy Normalized (CENS, 12 chroma) - 84 features
chroma_cqt: Constant-Q chromagram (12 chroma) - 84 features
chroma_stft: Chromagram (12 chroma) - 84 features
mfcc: Mel-frequency cepstrum (20 coefficients) - 140 features
rmse: Root-mean-square - 7 features
spectral_bandwidth: Spectral bandwidth - 7 features
spectral_centroid: Spectral centroid - 7 features
spectral_contrast: Spectral contrast (7 frequency bands) - 49 features
spectral_rolloff: Roll-off freque

In [51]:
# Prepare data
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_np.ravel()) #

# Split training data into training and temporary validation sets
X_train, X_temp, Y_train, Y_temp = train_test_split(x_train, y_train_encoded, test_size=0.4, random_state=42)

# Split the temporary validation set into validation and fake test set
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)  
X_real_test_scaled = scaler.transform(x_test) # real test set we don't have labels for

In [10]:
def split_features_by_type(X, feature_structure):
    """
    Splits the dataset into subsets based on the feature structure provided.

    :param X: numpy array, the dataset to be split (features only)
    :param feature_structure: dict, keys are feature names and values are the number of features of that type
    :return: dict of feature subsets
    """
    feature_subsets = {}
    start_idx = 0
    
    for feature_name, feature_count in feature_structure.items():
        end_idx = start_idx + feature_count
        feature_subsets[feature_name] = X[:, start_idx:end_idx]
        start_idx = end_idx
    
    return feature_subsets

# Define the feature structure
feature_structure = {
    'chroma_cens': 84,
    'chroma_cqt': 84,
    'chroma_stft': 84,
    'mfcc': 140,
    'rmse': 7,
    'spectral_bandwidth': 7,
    'spectral_centroid': 7,
    'spectral_contrast': 49,
    'spectral_rolloff': 7,
    'tonnetz': 42,
    'zcr': 7
}

# Boosting separately on feature subsets

### Load Data and Convert to DMatrix

In [52]:
train_subsets = split_features_by_type(X_train_scaled, feature_structure)
val_subsets = split_features_by_type(X_val_scaled, feature_structure)
test_subsets = split_features_by_type(X_test_scaled, feature_structure)

In [13]:
def objective(trial, X_sub, Y_sub):
    # Hyperparameters
    params = {
        'objective': 'multi:softmax',
        'num_class': 8,
        'max_depth': trial.suggest_int('max_depth', 3, 60),
        'eta': trial.suggest_float('eta', 0.01, 0.4),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'eval_metric': 'merror'  # Multiclass Classification Error
    }

    # Convert the subset dataset into DMatrix form
    dmatrix = xgb.DMatrix(X_sub, label=Y_sub)

    # Perform cross-validation
    cv_results = xgb.cv(params, dmatrix, num_boost_round=5000, nfold=5, stratified=True,
                        early_stopping_rounds=25, seed=42, verbose_eval=False)

    # Extract the minimum mean merror from the CV results
    min_mean_merror = cv_results['test-merror-mean'].min()

    return min_mean_merror

In [14]:
best_params_subsets = {}
validation_accuracies = {}

for feature_name, feature_count in feature_structure.items():
    print(f"Running study for feature subset: {feature_name}")
    
    # Prepare the data for this subset
    X_sub_train = train_subsets[feature_name]
    Y_sub_train = Y_train  # Y_train should be defined in your context

    def subset_objective(trial):
        return objective(trial, X_sub_train, Y_sub_train)

    study = optuna.create_study(direction='minimize', study_name=f"XGB_{feature_name}")
    study.optimize(subset_objective, n_trials=100)

    best_params_subsets[feature_name] = study.best_trial.params

    # # Merge train and validation subsets for final model retraining
    # X_sub_train_val_combined = np.vstack((X_sub_train, val_subsets[feature_name]))
    # Y_train_val_combined = np.concatenate((Y_train, Y_val))  # Assuming Y_val is defined
    # 
    # # Retrain the model on the combined training and validation set with the best parameters
    # dtrain_val_combined = xgb.DMatrix(X_sub_train_val_combined, label=Y_train_val_combined)
    # Retrain model with optimal parameters with more boosting rounds
    params = {
        'objective': 'multi:softmax',
        'num_class': 8,
        **best_params_subsets[feature_name],  # Unpack the best parameters
    }
    final_model = xgb.train(params, X_sub_train, num_boost_round=10_000) # dtrain_val_combined

    # Evaluate the final model on the validation set
    dval = xgb.DMatrix(val_subsets[feature_name], label=Y_val)
    preds = final_model.predict(dval)
    val_accuracy = accuracy_score(Y_val, preds)
    validation_accuracies[feature_name] = val_accuracy

    # Save the final model
    model_name = f'Models/XGBoost-Feature-Subsets/xgboost_{feature_name}_final.model'
    final_model.save_model(model_name)

    print(f"Validation accuracy for {feature_name}: {val_accuracy}")
    

[I 2024-03-13 23:14:32,121] A new study created in memory with name: XGB_chroma_cens


Running study for feature subset: chroma_cens


[W 2024-03-13 23:15:03,961] Trial 0 failed with parameters: {'max_depth': 16, 'eta': 0.18260583038077527, 'subsample': 0.8210097791906202, 'colsample_bytree': 0.8412711335976696} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_278/2774506645.py", line 12, in subset_objective
    return objective(trial, X_sub_train, Y_sub_train)
  File "/tmp/ipykernel_278/3102675667.py", line 17, in objective
    cv_results = xgb.cv(params, dmatrix, num_boost_round=5000, nfold=5, stratified=True,
  File "/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/xgboost/training.py", line 538, in cv
    booster.update(i, obj)
  File "/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/xgboost/training.py", line 229, in update
    fold.update(iteration, obj)
  File "/opt/saturncloud/envs/

KeyboardInterrupt: 

## Save Best Parameters 

In [ ]:
import json
from datetime import datetime
# Format the current date and time as a string
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Filenames with date and time
filename_best_params = f"best_params_subsets_{timestamp}.json"
filename_validation_accuracies = f"validation_accuracies_{timestamp}.json"

# Save best_params_subsets
with open(filename_best_params, 'w') as file:
    json.dump(best_params_subsets, file, indent=4)

# Save validation_accuracies
with open(filename_validation_accuracies, 'w') as file:
    json.dump(validation_accuracies, file, indent=4)


## Add, Commit, Push Tuning Results

In [ ]:
!git add .
!git commit -m "tuning completed"
!git push origin HEAD


## Load Results and Ensemble

In [12]:
import json
with open("validation_accuracies_2024-03-14_13-37-15.json", 'r') as file:
    validation_accuracies = json.load(file)

In [13]:
print(validation_accuracies)

{'chroma_cens': 0.3358333333333333, 'chroma_cqt': 0.37, 'chroma_stft': 0.38666666666666666, 'mfcc': 0.5541666666666667, 'rmse': 0.24333333333333335, 'spectral_bandwidth': 0.25916666666666666, 'spectral_centroid': 0.3025, 'spectral_contrast': 0.47, 'spectral_rolloff': 0.30583333333333335, 'tonnetz': 0.32166666666666666, 'zcr': 0.29083333333333333}


In [57]:
import xgboost as xgb
import os

models = {}  # To store the loaded models
model_path = "/Users/ts/Desktop/XGBoost-Feature-Subsets"  # Replace with your actual model path

# List all model files in the directory with a specific pattern
model_files = [f for f in os.listdir(model_path) if f.startswith('xgboost_') and f.endswith('_accuracy.model')]

for filename in model_files:
    parts = filename.split('_')
    feature_name = '_'.join(parts[1:-2])  # Join everything except the last two parts (accuracy value and 'model')
    model_filename = os.path.join(model_path, filename)
    model = xgb.Booster()
    model.load_model(model_filename)
    models[feature_name] = model

In [58]:
models

{'spectral_bandwidth': <xgboost.core.Booster at 0x2cb07bfa0>,
 'spectral_rolloff': <xgboost.core.Booster at 0x2cb07bd90>,
 'tonnetz': <xgboost.core.Booster at 0x2cb07b2e0>,
 'chroma_cens': <xgboost.core.Booster at 0x2cb07b550>,
 'rmse': <xgboost.core.Booster at 0x2cb07b490>,
 'spectral_contrast': <xgboost.core.Booster at 0x2cb07b520>,
 'chroma_cqt': <xgboost.core.Booster at 0x2cb07bd30>,
 'zcr': <xgboost.core.Booster at 0x2cb07bca0>,
 'chroma_stft': <xgboost.core.Booster at 0x2cb07b970>,
 'mfcc': <xgboost.core.Booster at 0x2caf05390>,
 'spectral_centroid': <xgboost.core.Booster at 0x2caf05180>}

## Simple Majority Voting

In [59]:
import numpy as np
from scipy.stats import mode

# Dictionary to store predictions from each model
predictions = {}

for feature_name, model in models.items():
    # Extract the test subset for this feature
    X_test_subset = test_subsets[feature_name]
    
    # Convert to DMatrix
    dtest = xgb.DMatrix(X_test_subset)
    
    # Predict using the model
    preds = model.predict(dtest)
    
    # Store predictions
    predictions[feature_name] = preds


In [60]:
# Assuming predictions are class labels and not probabilities

# Convert prediction values to int if necessary
for key in predictions:
    predictions[key] = predictions[key].astype(int)

# Stack predictions vertically to form a matrix where each column represents a model's predictions
pred_matrix = np.vstack(list(predictions.values()))

# Perform majority voting
final_predictions, _ = mode(pred_matrix, axis=0)

# Flatten array to get final class labels
final_predictions = final_predictions.flatten()


In [61]:
from sklearn.metrics import accuracy_score

# Assuming Y_test contains the true labels for the test data
ensemble_accuracy = accuracy_score(Y_test, final_predictions)
print(f"Ensemble accuracy: {ensemble_accuracy}")


Ensemble accuracy: 0.5341666666666667


## Weighted Voting

In [63]:
# Normalize validation accuracies to use as weights
total_accuracy = sum(validation_accuracies.values())
weights = {k: v / total_accuracy for k, v in validation_accuracies.items()}


In [64]:
# Initialize a dictionary to keep track of weighted votes for each class
weighted_votes = np.zeros((len(Y_test), len(np.unique(Y_test))))  # Assuming Y_test is available

for feature_name, preds in predictions.items():
    weight = weights[feature_name]
    for i, pred in enumerate(preds):
        weighted_votes[i, int(pred)] += weight

# Determine the final predictions based on weighted votes
final_predictions_weighted = np.argmax(weighted_votes, axis=1)
ensemble_accuracy_weighted = accuracy_score(Y_test, final_predictions_weighted)
print(f"Ensemble accuracy (weighted): {ensemble_accuracy_weighted}")


Ensemble accuracy (weighted): 0.5608333333333333


## Stacking

In [65]:
base_model_predictions_train = {}
base_model_predictions_test = {}

for feature_name, model in models.items():
    # Training data for stacking
    X_sub_train = train_subsets[feature_name]
    dtrain = xgb.DMatrix(X_sub_train)
    train_preds = model.predict(dtrain)
    
    # Test data for final evaluation
    X_sub_test = test_subsets[feature_name]
    dtest = xgb.DMatrix(X_sub_test)
    test_preds = model.predict(dtest)
    
    base_model_predictions_train[feature_name] = train_preds
    base_model_predictions_test[feature_name] = test_preds
# Assuming predictions are class labels; adjust if they are probabilities
X_train_stacked = np.column_stack(list(base_model_predictions_train.values()))
X_test_stacked = np.column_stack(list(base_model_predictions_test.values()))
from sklearn.linear_model import LogisticRegression

# Initialize and train the meta-learner
meta_learner = LogisticRegression(max_iter=10_000)
meta_learner.fit(X_train_stacked, Y_train)  # Assuming Y_train is your target variable
from sklearn.metrics import accuracy_score

# Make predictions
final_predictions_stacked = meta_learner.predict(X_test_stacked)

# Evaluate accuracy
stacked_accuracy = accuracy_score(Y_test, final_predictions_stacked)  # Assuming Y_test is available
print(f"Stacked model accuracy: {stacked_accuracy}")


Stacked model accuracy: 0.18583333333333332


In [66]:
from sklearn.ensemble import GradientBoostingClassifier

meta_learner_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=50)
meta_learner_gb.fit(X_train_stacked, Y_train)
final_predictions_stacked_gb = meta_learner_gb.predict(X_test_stacked)

stacked_accuracy_gb = accuracy_score(Y_test, final_predictions_stacked_gb)
print(f"Stacked model accuracy with Gradient Boosting meta-learner: {stacked_accuracy_gb}")


Stacked model accuracy with Gradient Boosting meta-learner: 0.5241666666666667


In [67]:
from sklearn.ensemble import RandomForestClassifier

meta_learner_rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
meta_learner_rf.fit(X_train_stacked, Y_train)
final_predictions_stacked_rf = meta_learner_rf.predict(X_test_stacked)

stacked_accuracy_rf = accuracy_score(Y_test, final_predictions_stacked_rf)
print(f"Stacked model accuracy with Random Forest meta-learner: {stacked_accuracy_rf}")


Stacked model accuracy with Random Forest meta-learner: 0.4708333333333333


In [68]:
from sklearn.svm import SVC

meta_learner_svm = SVC(kernel='linear', C=1)
meta_learner_svm.fit(X_train_stacked, Y_train)
final_predictions_stacked_svm = meta_learner_svm.predict(X_test_stacked)

stacked_accuracy_svm = accuracy_score(Y_test, final_predictions_stacked_svm)
print(f"Stacked model accuracy with SVM meta-learner: {stacked_accuracy_svm}")


Stacked model accuracy with SVM meta-learner: 0.18583333333333332


In [69]:
from sklearn.neural_network import MLPClassifier

meta_learner_nn = MLPClassifier(hidden_layer_sizes=(50, ), activation='relu', solver='adam', max_iter=500)
meta_learner_nn.fit(X_train_stacked, Y_train)
final_predictions_stacked_nn = meta_learner_nn.predict(X_test_stacked)

stacked_accuracy_nn = accuracy_score(Y_test, final_predictions_stacked_nn)
print(f"Stacked model accuracy with Neural Network meta-learner: {stacked_accuracy_nn}")


Stacked model accuracy with Neural Network meta-learner: 0.1975


## Using Validation Set

In [70]:
# Initialize containers for meta-features
X_meta_train = []  # For storing meta-features of the training split
X_meta_val = []    # For storing meta-features of the validation set

# Generate predictions (meta-features) for each feature subset
for feature_name in feature_structure.keys():
    # Training split predictions
    X_sub_train = train_subsets[feature_name]
    dtrain = xgb.DMatrix(X_sub_train)
    train_preds = models[feature_name].predict(dtrain)
    X_meta_train.append(train_preds)
    
    # Validation set predictions
    X_sub_val = val_subsets[feature_name]
    dval = xgb.DMatrix(X_sub_val)
    val_preds = models[feature_name].predict(dval)
    X_meta_val.append(val_preds)

# Convert lists to numpy arrays for easier handling
X_meta_train = np.column_stack(X_meta_train)
X_meta_val = np.column_stack(X_meta_val)


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the meta-learner
meta_learner = LogisticRegression(max_iter=1000)

# Train the meta-learner on the training meta-features
meta_learner.fit(X_meta_train, Y_train)  # Y_train should match the part of data used to generate X_meta_train

# Validate the meta-learner on the validation meta-features
val_predictions = meta_learner.predict(X_meta_val)
val_accuracy = accuracy_score(Y_val, val_predictions)  # Assuming Y_val is correctly ordered and matches X_meta_val

print(f"Validation accuracy of the meta-learner: {val_accuracy}")


Validation accuracy of the meta-learner: 0.17833333333333334


In [31]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize Gradient Boosting meta-learner
gb_meta_learner = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=3)

# Train the meta-learner on the stacked training data
gb_meta_learner.fit(X_meta_train, Y_train)

# Validate the meta-learner on the validation meta-features
val_predictions_gb = gb_meta_learner.predict(X_meta_val)
val_accuracy_gb = accuracy_score(Y_val, val_predictions_gb)

print(f"Validation accuracy with Gradient Boosting meta-learner: {val_accuracy_gb}")


Validation accuracy with Gradient Boosting meta-learner: 0.5


In [32]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree meta-learner
dt_meta_learner = DecisionTreeClassifier(max_depth=5)

# Train the meta-learner
dt_meta_learner.fit(X_meta_train, Y_train)

# Validate the meta-learner
val_predictions_dt = dt_meta_learner.predict(X_meta_val)
val_accuracy_dt = accuracy_score(Y_val, val_predictions_dt)

print(f"Validation accuracy with Decision Tree meta-learner: {val_accuracy_dt}")


Validation accuracy with Decision Tree meta-learner: 0.2


In [33]:
from sklearn.neural_network import MLPClassifier

# Initialize Neural Network meta-learner
nn_meta_learner = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=500)

# Train the meta-learner
nn_meta_learner.fit(X_meta_train, Y_train)

# Validate the meta-learner
val_predictions_nn = nn_meta_learner.predict(X_meta_val)
val_accuracy_nn = accuracy_score(Y_val, val_predictions_nn)

print(f"Validation accuracy with Neural Network meta-learner: {val_accuracy_nn}")


Validation accuracy with Neural Network meta-learner: 0.165


In [43]:
import numpy as np

# Dictionary to store probability predictions from each model
prob_predictions = {}

for feature_name, model in models.items():
    # Extract the test subset for this feature
    X_test_subset = test_subsets[feature_name]
    
    # Convert to DMatrix
    dtest = xgb.DMatrix(X_test_subset)
    
    # Predict using the model to get probabilities
    # probs = model.predict(dtest, ntree_limit=model.best_ntree_limit, validate_features=False)
    probs = model[:model.best_ntree_limit].predict(dtest, validate_features=False)

    # Store probability predictions
    prob_predictions[feature_name] = probs

# Now, `prob_predictions` contains class probabilities instead of labels


In [44]:
prob_predictions

{'chroma_cens': array([4., 4., 4., ..., 6., 7., 4.], dtype=float32),
 'chroma_cqt': array([2., 6., 1., ..., 3., 7., 2.], dtype=float32),
 'chroma_stft': array([4., 7., 1., ..., 0., 3., 4.], dtype=float32),
 'mfcc': array([2., 1., 0., ..., 3., 3., 4.], dtype=float32),
 'rmse': array([1., 7., 6., ..., 3., 6., 1.], dtype=float32),
 'spectral_bandwidth': array([5., 2., 7., ..., 5., 6., 1.], dtype=float32),
 'spectral_centroid': array([2., 6., 7., ..., 3., 1., 1.], dtype=float32),
 'spectral_contrast': array([2., 6., 1., ..., 3., 6., 4.], dtype=float32),
 'spectral_rolloff': array([2., 4., 6., ..., 0., 7., 4.], dtype=float32),
 'tonnetz': array([1., 4., 3., ..., 3., 3., 4.], dtype=float32),
 'zcr': array([2., 6., 4., ..., 3., 3., 1.], dtype=float32)}

In [46]:
# Assuming predictions are class labels and not probabilities

# Convert prediction values to int if necessary
for key in prob_predictions:
    prob_predictions[key] = prob_predictions[key].astype(int)

# Stack predictions vertically to form a matrix where each column represents a model's predictions
pred_matrix = np.vstack(list(prob_predictions.values()))

# Perform majority voting
final_predictions, _ = mode(pred_matrix, axis=0)

# Flatten array to get final class labels
final_predictions = final_predictions.flatten()

from sklearn.metrics import accuracy_score

# Assuming Y_test contains the true labels for the test data
ensemble_accuracy = accuracy_score(Y_test, final_predictions)
print(f"Ensemble accuracy: {ensemble_accuracy}")

Ensemble accuracy: 0.5475


In [47]:
prob_predictions = {}

for feature_name, model in models.items():
    X_test_subset = test_subsets[feature_name]
    
    # Convert to DMatrix
    dtest = xgb.DMatrix(X_test_subset)
    
    # Predict to get probabilities
    probs = model.predict(dtest, output_margin=False)
    
    prob_predictions[feature_name] = probs


In [48]:
prob_predictions

{'chroma_cens': array([4., 4., 4., ..., 6., 7., 4.], dtype=float32),
 'chroma_cqt': array([2., 6., 1., ..., 3., 7., 2.], dtype=float32),
 'chroma_stft': array([4., 7., 1., ..., 0., 3., 4.], dtype=float32),
 'mfcc': array([2., 1., 0., ..., 3., 3., 4.], dtype=float32),
 'rmse': array([1., 7., 6., ..., 3., 6., 1.], dtype=float32),
 'spectral_bandwidth': array([5., 2., 7., ..., 5., 6., 1.], dtype=float32),
 'spectral_centroid': array([2., 6., 7., ..., 3., 1., 1.], dtype=float32),
 'spectral_contrast': array([2., 6., 1., ..., 3., 6., 4.], dtype=float32),
 'spectral_rolloff': array([2., 4., 6., ..., 0., 7., 4.], dtype=float32),
 'tonnetz': array([1., 4., 3., ..., 3., 3., 4.], dtype=float32),
 'zcr': array([2., 6., 4., ..., 3., 3., 1.], dtype=float32)}